In [1]:
import numpy as np

### [Gaussian elimination](https://en.wikipedia.org/wiki/Gaussian_elimination)

In [2]:
def gaussian(A,b):
    """
    Arguments:
    A -- numpy.ndarray -- given system of coeffitients
    x -- numpy.ndarray -- unknown vector X 
    b -- vector b
    Output:
    x -- root of the system of equations A*x=b
    """
    n,matr = len(b),np.hstack([A,b.reshape(-1,1)])
    if np.linalg.det(A)==0: 
        return -1
    
    #direct move
    for i in range(n):
        a = matr[i]
        for j in range(i+1,n):
            b = matr[j]
            m = a[i]/b[i]
            matr[j] = a - m*b
            
    #reversed move
    for i in range(n-1,-1,-1):
        matr[i]= matr[i]/matr[i,i]
        a = matr[i]
        for j in range(i-1,-1,-1):
            b = matr[j]
            m = a[i]/b[i]
            matr[j] = a - m*b

    return matr[:,3]

In [3]:
A = np.array([[1, 2, 3], [3, 5, 7], [1, 3, 4]], dtype='float')
b = np.array([3, 0, 1])
print(gaussian(A,b))
t1,t2 = gaussian(A,b),np.linalg.solve(A,b)
t1 = np.round(t1,10)
t2 = np.round(t2,10)
np.testing.assert_array_equal(t1,t2)

[ -4. -13.  11.]


### [Cholesky decomposition](https://en.wikipedia.org/wiki/Cholesky_decomposition) a.k.a. Least Squares Method

$Ax=b$<br>

If A is symmetric & [positive-definite](https://en.wikipedia.org/wiki/Definiteness_of_a_matrix):<br>
Let $A := L*L^T$<br>

$L * y=b $ - forward substitution <br>
$L^T*x=y $  - back substitution<br><br>

LDLT decomposition:<br>

Let $A:= L * D * L^T$<br>
$L * y = b $<br>
$DL* x = y $

In [4]:
def ldl_decomposition(A):
    L_cholesky = np.linalg.cholesky(A)
    S = L_cholesky.diagonal()
    D = S**2
    S_inv = np.linalg.inv(np.diagflat(S))
    L = np.dot(L_cholesky,S_inv)
    return L,D
def cholesky(A,b):
    if np.array_equal(A,A.T):
        try:
            L = np.linalg.cholesky(A)
            y = np.linalg.solve(L,b)
            x = np.linalg.solve(L.T,y)
        except np.linalg.LinAlgError:
            return "Unsutable matrix"
    return x

In [5]:
T = np.array([[4,12,-16],[12,37,-43],[-16,-43,98]])
A,b = np.array([[25,15,-5],[15,18,0],[-5,0,11]]),np.array([-2,0.25,10.2])
cholesky(A,b)
ldl_decomposition(T)

(array([[ 1.,  0.,  0.],
        [ 3.,  1.,  0.],
        [-4.,  5.,  1.]]), array([4., 1., 9.]))

### [Tridiagonal matrix algorithm](https://en.wikipedia.org/wiki/Tridiagonal_matrix_algorithm)

In [6]:
def get_coeff_vect(array):
    """
    Arguments:
    ----------
    array -- array of shape(n,n)
    returns triagonals a,b,c
    Output:
    a,b,c -- tridiagonals of an array
    """
    n = len(array)
    ind1,ind2 = [i for i in range(1,n)],[i-1 for i in range(1,n)]
    a = np.copy(array[ind1,ind2])
    b = np.copy(array[[i for i in range(n)],[i for i in range(n)]])
    c = np.copy(array[ind2,ind1])
    return a,b,c


def tridiagonal_matr_algo(arr,f):
    """
    Arguments:
    ----------
    arr -- array of shape(n,n)
    f -- vector of shape(n,)
    local_coeff -- bool, uses slighty different calculations
    Output:
    x -- solution of system of equations A*x = f
    """
    n,f= len(arr),np.copy(f)  
    a,b,c = get_coeff_vect(arr)
    
    # forward step
    for i in range(1,n):
        w = a[i-1]/b[i-1]
        b[i] -= w*c[i-1]
        f[i] -= w*f[i-1]

    #backward step
    x = b
    x[-1] = f[-1]/b[-1]
    for i in range(n-2,-1,-1):
        x[i]=(f[i]-c[i]*x[i+1])/(b[i])
    
    return x   

In [7]:
A,d = np.array([[-4,1,0,0],[1,-4,-1,0],[0,1,-4,1],[0,0,1,-4]],dtype=float),np.array([-6,-4,-4,-6],dtype=float)

In [8]:
C,d = np.array([[-4,1,0,0],[1,-4,1,0],[0,1,-4,1],[0,0,1,-4]],dtype=float),np.array([-6,-4,-4,-6],dtype=float)
print("TMA: {0}".format(tridiagonal_matr_algo(C,d)))
print("Standard Gaussian solver: {0}".format(np.linalg.solve(C,d)))

TMA: [2. 2. 2. 2.]
Standard Gaussian solver: [2. 2. 2. 2.]
